In [1]:
'''
This is the main tool. I have tried defining it into
its own function, but loading model checkpoints does 
not seem to allow that. Nonetheless, it can be easily
controlled from here

This extracts the inner layers of a trained model, 
first by loading saved checkpoint of model
then running a forward pass through the model

As soon as this is placed inside a function,
the ability to load the model is lost for some reason.

'''

import torch
import schnetpack as spk
import math
from schnetpack.datasets import QM9
import os

import numpy as np
from numpy import savetxt


qm9data = QM9('../../data/QM9/qm9.db', download=False, remove_uncharacterized=True)
split_file='../../data/trained_models/schnet_qm9energy_1000030/split.npz'
checkpoint_path = '../../data/trained_models/schnet_qm9energy_1000030/trained.pth'
n_atom_basis=30
n_filters=30
n_gaussians=20
n_interactions=3
cutoff = 4.


number_inputs=5000
name_data = 'rep'



In [2]:
def hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global layer
    #Update the embedding_output variable to be equal to our output tensor
    layer=out_tensor 

# For the QM9 dataset

In [ ]:
# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file=split_file)

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)

# Define SchNet representation model

schnet = spk.representation.SchNet(
n_atom_basis=n_atom_basis, n_filters=n_filters, n_gaussians=n_gaussians, n_interactions=n_interactions,
cutoff=cutoff , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=n_filters, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
load_checkpoint = torch.load(checkpoint_path)


#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint)


#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

datao = np.zeros((1,30))
datahae = np.zeros((1))
dataoae = np.zeros((1))
datah = np.zeros((1,30))
data = np.zeros((1))
datac = np.zeros((1,30))
datacae = np.zeros((1))
datan = np.zeros((1,30))
datanae = np.zeros((1))
for idx in range(number_inputs):
    at, props = qm9data.get_properties(idx)
    inputs = converter(at)
    
    layer = None
    
    model.representation.embedding.register_forward_hook(hook)
    model(inputs)
    
    emb = layer.clone()
    emb = layer.detach().numpy()
    
    layer = None
    
    model.representation.interactions[0].register_forward_hook(hook)
    
    model(inputs)
    int0 = layer.clone()
    int0 = int0.detach().numpy()    

    layer = None
    
    model.representation.interactions[1].register_forward_hook(hook)
    
    model(inputs)
    int1 = layer.clone()
    int1 = int1.detach().numpy()   
    
    layer = None 
    
    model.representation.interactions[2].register_forward_hook(hook)
    
    model(inputs)   
    
    int2 = layer.clone()
    int2 = int2.detach().numpy()   

    rep = emb+int0+int1+int2
    
    number_atoms = len(props['_positions'])
    rows = np.zeros((number_atoms,30))
    for i in range(number_atoms):
        for j in range(30):
            rows[i][j] = rep[0][i][j]

    from schnetpack.atomistic.output_modules import yi
    
    yi=yi.detach().numpy()
    #save the vector of every oxygen atom encountered
    
    
    for i in range(number_atoms):
        if props['_atomic_numbers'][i] == 8:
            datao = np.vstack((datao,rows[i]))
            dataoae = np.vstack((dataoae,yi[0][i]))
        if props['_atomic_numbers'][i] == 1:
            datah = np.vstack((datah,rows[i]))    
            datahae = np.vstack((datahae,yi[0][i]))
        if props['_atomic_numbers'][i] == 6:
            datac = np.vstack((datac,rows[i]))    
            datacae = np.vstack((datacae,yi[0][i]))
        if props['_atomic_numbers'][i] == 7:
            datan = np.vstack((datan,rows[i]))    
            datanae = np.vstack((datanae,yi[0][i]))
            
save_path = '../../data/schnet/%s/'  %(str(number_inputs))
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
    
savetxt(save_path+name_data+'O.csv',datao,delimiter=',')
savetxt(save_path+name_data+'H.csv',datah,delimiter=',')
savetxt(save_path+name_data+'Hae.csv',datahae,delimiter=',')
savetxt(save_path+name_data+'Oae.csv',dataoae,delimiter=',')
savetxt(save_path+name_data+'C.csv',datac,delimiter=',')
savetxt(save_path+name_data+'Cae.csv',datacae,delimiter=',')
savetxt(save_path+name_data+'N.csv',datan,delimiter=',')
savetxt(save_path+name_data+'Nae.csv',datanae,delimiter=',')